Dependencies

In [14]:
import SimpleITK as sitk
import torch 
import numpy as np
from models import unet

Load Model

In [12]:
model = unet.UNet(in_channels=1,num_classes=19)
#setting cpu for laptop inf testing
model.load_state_dict(torch.load("C:/Users/Konstantinos/Desktop/Spider Inference Models Test/model_20240319_104349_2", map_location=torch.device('cpu')))

model.eval()

c:\Github Repositories\LabML Unet\spider-seg-e19005\models\unet.py:213: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(m.weight)
c:\Github Repositories\LabML Unet\spider-seg-e19005\models\unet.py:214: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias, 0)


UNet(
  (conv_final): Conv2d(64, 19, kernel_size=(1, 1), stride=(1, 1))
  (down_convs): ModuleList(
    (0): DownConv(
      (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): DownConv(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): DownConv(
      (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (3): DownConv(
      (conv1): Conv2d(256, 512, kernel_size=(3,

Test In MRI from images removed from training sets  

In [33]:
in_mha = sitk.ReadImage("out_inference/132_t1.mha")

in_arr = sitk.GetArrayFromImage(in_mha)

if(in_arr.shape[0] > in_arr.shape[1] or  in_arr.shape[0] > in_arr.shape[2]):
    in_arr = np.transpose(in_arr, (2, 0, 1))

print(in_arr.shape)

#image normalisation
for idx in range (in_arr.shape[0]):
    #print(in_arr[idx, :, :].shape)
    idx_slice = in_arr[idx, : , :]
    if (idx == 0):
        image_max = np.max(idx_slice)
        image_min = np.min(idx_slice)
    else:
        if(np.max(idx_slice) > image_max):
            image_max = np.max(idx_slice)
        if(np.min(idx_slice) < image_min):
            image_min = np.min(idx_slice)

print("max", image_max, "min", image_min)

out_arr = np.empty_like(in_arr)

#convert each slice to tensor run inf on each slice and save in new np arr
for idx in range(in_arr.shape[0]):
    in_slice = in_arr[idx, :, :]
    in_tensor = torch.from_numpy(in_slice)
    in_tensor = (in_tensor - image_min) / (image_max - image_min)
    in_tensor = in_tensor.to(torch.float32)
    in_tensor = in_tensor.unsqueeze(0)
    print(in_tensor.shape)

    out_tensor = model(in_tensor)

    #out_arr = out_tensor.numpy()


    


(15, 320, 320)
max 3096 min -1000
torch.Size([1, 320, 320])


RuntimeError: Given groups=1, weight of size [512, 1024, 3, 3], expected input[1, 512, 80, 40] to have 1024 channels, but got 512 channels instead

Split the image into slices in loop and run inference model eval 